In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
import os,sys
sys.path.append('./misc/lib/python3.7/site-packages')

import math
import numpy as np
import requests
import ipywidgets as widgets
import matplotlib.pyplot as plt
from IPython.display import display, display_markdown
from ipywidgets import Layout, HTML
from pathlib import Path

from scipy import spatial

NGL_DEF = False
try:
    import nglview as nv
    NGL_DEF = True
except:
    NGL_DEF = False
    

import parmed as pmd
import re

from scipy.ndimage import gaussian_filter

np.set_printoptions(precision=8)
np.set_printoptions(suppress=True)


HTMLButtonPrompt = '''<html>
<head>
<meta name="viewport" content="width=device-width, initial-scale=1">
</head>
<body>
<a href="{link}" target="_blank" >
<button class="p-Widget jupyter-widgets jupyter-button widget-button mod-warning" style="width:100px; background-color:#E9E9E9; font-size:10pt; color:black">{text}</button>
</a>
</body>
</html>
'''

HTMLDeadPrompt = '''<html>
<head>
<meta name="viewport" content="width=device-width, initial-scale=1">
</head>
<body>
<button class="p-Widget jupyter-widgets jupyter-button widget-button mod-warning" style="width:100px; background-color:#E9E9E9; font-size:10pt; color:#D2D2D2">{text}</button>
</body>
</html>
'''

forbidden_strings = ["..", "/", "\\", " ", "~"]

In [3]:
sys.path.append('./main/')

import pigment
import tresp

DATADIR = 'data'

In [4]:
# These names are considered essential to all Chl/BChl molecules
# Without these, we can do no calculations. 
ChlBaseNames = []
for ring in ['A', 'B', 'C', 'D']:
    for at in ['N', 'C1', 'C2', 'C3', 'C4', 'CH']:
        ChlBaseNames.append(at+ring)
ChlBaseNames.append('CAD')
ChlBaseNames.append('CBD')

# These are the atoms of the phytol tail
PhytolNames = []
for n in range(1, 21):
    PhytolNames.append('C'+str(n))
    
# These atoms are 
ChlOptNames = ['CAA', 'CBA', 'CGA', 'O1A', 'O2A', 'CMA', 'CMB', 'CAB', 'CMC', 'CAC', 'CBC', 'CMD', 'OBD'] + PhytolNames

# The tetpy class describes common tetrapyrrole pigments
tetpy = pigment.category('tetrapyrrole', ChlBaseNames)


# These atoms can be used to distinguish between different pigment types
# Note: 
#    The heavy atoms of Chl a, Chl c1, Chl c2, and BChl g are identical (excluding the phytol tail)
#    The heavy atoms of BChl a and BChl b are identical
CLANames = ['MG', 'CBB', 'CGD', 'O1D', 'O2D', 'CED'] + ChlOptNames
CLBNames = ['MG', 'CBB', 'CGD', 'O1D', 'O2D', 'CED', 'OMC']  + ChlOptNames
CLDNames = ['MG',        'CGD', 'O1D', 'O2D', 'CED',        'OBB'] + ChlOptNames
CLFNames = ['MG', 'CBB', 'CGD', 'O1D', 'O2D', 'CED',               'OMB']  + ChlOptNames
BCANames = ['MG', 'CBB', 'CGD', 'O1D', 'O2D', 'CED',        'OBB'] + ChlOptNames
BCBNames = ['MG', 'CBB', 'CGD', 'O1D', 'O2D', 'CED',        'OBB'] + ChlOptNames
BCCNames = ['MG', 'CBB',                                    'OBB',        'CIB'] + ChlOptNames
BCDNames = ['MG', 'CBB',                                    'OBB',               'CND'] + ChlOptNames
BCENames = ['MG', 'CBB',                             'OMC', 'OBB',        'CIB', 'CND'] + ChlOptNames
BCFNames = ['MG', 'CBB',                             'OMC', 'OBB',               'CND'] + ChlOptNames
BCGNames = ['MG', 'CBB', 'CGD', 'O1D', 'O2D', 'CED'] + ChlOptNames

# The corresponding Pheo names are the same, excluding the first entry (MG)
PHANames = CLANames[1:]
PHBNames = CLBNames[1:]
BPANames = BCANames[1:]

CLA = pigment.species(
    'Chl a',
    'CLA',
    tetpy,
    CLANames,
    4.3e-18 # statC*cm
)

CLB = pigment.species(
    'Chl b',
    'CLB',
    tetpy,
    CLBNames, 
    3.60e-18
)

CLD = pigment.species(
    'Chl d',
    'CLD',
    tetpy,
    CLDNames, 
    0.0
)

CLF = pigment.species(
    'Chl f',
    'CLF',
    tetpy,
    CLFNames,
    0.0
)

BCA = pigment.species(
    'BChl a',
    'BCA',
    tetpy,
    BCANames, 
    5.477e-18
)

BCB = pigment.species(
    'BChl b',
    'BCB',
    tetpy,
    BCBNames, 
    0.0
)

BCC = pigment.species(
    'BChl c',
    'BCC',
    tetpy,
    BCCNames,
    0.0
)

BCD = pigment.species(
    'BChl d',
    'BCD',
    tetpy,
    BCDNames, 
    0.0
)

BCE = pigment.species(
    'BChl e',
    'BCE',
    tetpy,
    BCENames, 
    0.0
)

BCF = pigment.species(
    'BChl f',
    'BCF',
    tetpy,
    BCFNames, 
    0.0
)

BCG = pigment.species(
    'BChl g',
    'BCG',
    tetpy,
    BCGNames, 
    0.0
)

PHA = pigment.species(
    'Pheo a',
    'PHA',
    tetpy,
    PHANames,
    3.50e-18
)

UNK = pigment.species(
    'Unknown',
    'UNK',
    tetpy,
    [],
    0.0
)


TetPyList = [
    CLA, CLB, #CLD, CLF, 
    BCA, #BCB, BCC, BCD, BCE, BCF, BCG,
    PHA, 
]

In [5]:
# Global variables:

# Main ParmEd structure
struc = pmd.structure.Structure()

# File name frm which struc was loaded
struc_fname = ''

# Representation list for main structure
mainreps = list()

# Representation list for dipoles
dipreps = list()

# List of chains in struc
ChainList = []

# List of identified pigments
PigList = []

##################################################################
# Main frame layout:
##################################################################

# mainbox is the top widget. It contains:
#  pdbview -- an NGLWidget used to display the loaded structures
#  mainacc -- an accordion widget containing the following VBoxes:
#    strucbox -- contains the "Load" interface
#    selbox -- contains the "Select" interface
#    writebox -- contains the "Write PDB" interface
#    excbox -- contains the "Prepare Exciton Model" interface
#    mdbox -- contains the "Prepare MD Model" interface

# Structure viewer:
if NGL_DEF:
    pdbview = nv.NGLWidget()
    pdbview._set_size('500px', '500px')
    pdbview.camera = 'orthographic'
else:
    pdbview = widgets.HTML(value='<p style=\"text-align:center; font-size:20px\"><br><br>Install NGLView library<br>to view structures.</p>', 
                          layout=widgets.Layout(width='500px', height='500px'))


##################################################################
# strucbox: Interface for loading molecular structures
##################################################################

# pdbid: Text entry box for PDB ID 
# pdbidlbl: Label for pdbid
# pdbfetch: Button to fetch PDB from the RCSB databank
# pdbup: Button to upload PDB file
# pdbuplbl: Label for pdbup


pdbid = widgets.Text(
    value='2DRE',
    placeholder='',
    layout = widgets.Layout(width='1.5cm'),
    disabled=False
)

pdbidlbl = widgets.Label(value='Enter PDB ID:', layout=Layout(width='2.5cm'))

pdbfetch = widgets.Button(
    description='Fetch',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to display the pdb file',
    layout = widgets.Layout(width='2.25cm'),
    icon='' # (FontAwesome names without the `fa-` prefix)
)

pdbup = widgets.FileUpload(
    accept='.pdb, .gro',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False  # True to accept multiple files upload else False
)

pdbuplbl = widgets.Label(value='Or upload file:', layout=Layout(width='2.5cm'))

# Executed with pdbfetch is clicked
def pdbfetch_onclick(b):

    url = 'http://files.rcsb.org/download/' + pdbid.value + '.pdb'
    r = requests.get(url, allow_redirects=True)
    if(r.status_code!=200):
        print('Invalid PDB code. Please try again.')
    else:
        fname = DATADIR + '/pdb/'+pdbid.value+'.pdb'
        wfd = open(fname, 'wb')
        wfd.write(r.content)
        wfd.close()
        init_struc(fname)

# Executed when pdbup is clicked
def pdbup_on_value_change(change):
    
    for item in pdbup.value:
            fname = item
        
    with open(DATADIR + "/pdb/"+fname, "wb") as fp:
        fp.write(pdbup.value[fname]["content"])
    fp.close()
    init_struc(DATADIR + "/pdb/"+fname)
    
#     out = !{'ls ' + DATADIR + '/pdb/'}
#     for line in out:
#         print(line)
    
#     if os.path.isfile(DATADIR + '/pdb/' + fname):
#         print('Using stored local structure')
#         init_struc(DATADIR + "/pdb/"+fname)
#     else:
#         with open(DATADIR + "/pdb/"+fname, "wb") as fp:
#             fp.write(pdbup.value[fname]["content"])
#         fp.close()
#         init_struc(DATADIR + "/pdb/"+fname)
        
        
pdbup.observe(pdbup_on_value_change, 'value')


rundrop = widgets.Dropdown(
    options=['pdb2gmx', 'em', 'nvt'],
    value = 'pdb2gmx',
    description='Or pick',
    disabled=False,
    layout=Layout(width='3cm')
)
rundrop.style.description_width='1.1cm'
def rundrop_on_value_change(change):
    refresh_grolist(0)
rundrop.observe(rundrop_on_value_change, 'value')

grodrop = widgets.Dropdown(
    options=[],
    value = None,
    description='run:',
    disabled=False,
    layout=Layout(width='4cm')
)
grodrop.style.description_width='0.6cm'

def refresh_grolist(b):
    run = rundrop.value
    flist = !{"ls " + DATADIR + "/md/*/"+run+"/"+run+".pdb"}
    grolist = []
    for file in flist:
        prefix = file.split('/')[2]
        grolist.append(prefix)
    grodrop.options = grolist
    
# Executed when groload is clicked
def groload_onclick(b):
    prefix = grodrop.value
    if prefix!=None and prefix!='*' and len(prefix)>0:
        run = rundrop.value
        if run=='nvt':
            fname = DATADIR + "/md/"+prefix+"/"+run+"/traj.pdb"
        else:
            fname = DATADIR + "/md/"+prefix+"/"+run+"/"+run+".pdb"
        init_struc(fname)
        add_charges(DATADIR + "/md/"+prefix+"/"+run+"/charges.txt")
    
def add_charges(fname):
    global struc
    charges = np.loadtxt(fname)
    if len(charges)==len(struc.atoms):
        for n in range(0, len(struc.atoms)):
            atom = struc.atoms[n]
            atom.charge = charges[n]
            opts = list(siteselect.options)
            if opts.count('TrESP')==0:
                opts.append('TrESP')
                siteselect.options = opts
    else:
        print('Error loading charges from file ' + fname)
        print('Number of charges did not match the number of atoms in structure.')
        opts = list(siteselect.options)
        if opts.count('TrESP'):
            opts.remove('TrESP')
        siteselect.options = opts
        return -1
    
gro_refresh = widgets.Button(description='Refresh List')
gro_refresh.on_click(refresh_grolist)
refresh_grolist(0)

gro_load = widgets.Button(description='Load')
gro_load.on_click(groload_onclick)

strucbox = widgets.VBox([
    widgets.HBox([pdbidlbl, pdbid, pdbfetch]), 
    widgets.HBox([pdbuplbl, pdbup]),
    widgets.HBox([rundrop, grodrop]),
    widgets.HBox([gro_load, gro_refresh]),
])

# For now the displayed dipole structure is completely
# independent of the classified pigments. 
def build_dipstruc(struc):
    
    dipstruc = pmd.structure.Structure()
    
    dipcoords = []
    porphlist = list()
    porphtxt = '('
    PorphAts = ['NA', 'NB', 'NC', 'ND']
    for res in struc.residues:

        # Check if it's a porphyrrin
        foundNs = np.zeros((len(PorphAts),))
        patnums = np.zeros((len(PorphAts),), dtype='int')
        for at in res:
            for n in range(0, len(PorphAts)):
                if at.name==PorphAts[n]:
                    foundNs[n] = 1
                    patnums[n] = at.idx
        
        # If we located all four ring N atoms
        if np.sum(foundNs)==4:
            cenvec = 0.5*(struc.coordinates[patnums[1]] + struc.coordinates[patnums[3]])
            dipvec = (struc.coordinates[patnums[3]] - struc.coordinates[patnums[1]])
            vStart = cenvec - 1.25*dipvec
            vStop = cenvec + 1.25*dipvec
            Nats = 10
            for a in range(0, Nats+1):
                xyz = (float(a)/float(Nats))*vStart + (1 - float(a)/float(Nats))*vStop
                dipstruc.add_atom(pmd.topologyobjects.Atom(name='N'), 'Dip', res.idx, chain=res.chain)
                dipcoords.append(xyz)

            porphlist.append(res.idx)
            if len(porphtxt)>1:
                porphtxt += ' OR '
            porphtxt += str(res.idx+1)
    porphtxt += ')'

    # If any porphyrrins have been located, add the coordinates to the dipole list. 
    if len(dipcoords)>0:
        dipstruc.coordinates = np.array(dipcoords)
    else: 
        porphtxt = ""
        
    return dipstruc, porphtxt
    

# init_struc() loads a structure from the provided file name
# and initializes the structure view representations and chainlists
#
# Outside references:
#   sets pigbox.children and chainbox.children each time a new structure is loaded
#
# Relies on environment variables:
#   UNK
#   tetpy
#   TetPyList
def init_struc(fname):

    global mainreps
    global dipreps
    global struc
    global struc_fname
    global ChainList
    global chainbox
    global writebt
    global pigbox
    global PigList
    
    if NGL_DEF:
        # Clear pdbview stage
        while len(pdbview._ngl_component_ids)>0:
            pdbview.remove_component(pdbview._ngl_component_ids[0])
    
    # Reset rep and chain lists
    mainreps = list()
    dipreps = list()
    
    # Reset MD run button
    #mdrun_button.value = HTMLDeadPrompt.format(text='Simulate')
    
    # Reset site-energy-model list. If we load charges, these will be updated later. 
    opts = list(siteselect.options)
    if opts.count('TrESP'):
        opts.remove('TrESP')
    siteselect.options = opts
    
    struc = pmd.load_file(fname)
    struc_fname = fname
    dipstruc, porphtxt = build_dipstruc(struc)
    
    ChainList = []
    for res in struc.residues:
        # If the chain is not already listed, add it
        if ChainList.count(res.chain)==0:
            ChainList.append(res.chain)
    
    chaintxt = '('
    for chain in ChainList:
        if len(chaintxt)>1:
            chaintxt += ' OR '
        chaintxt += ':' + chain
    chaintxt += ')'

    if NGL_DEF:
        pdbview.add_trajectory(struc)
        mainreps = list()
        mainreps.append({"type": "cartoon", "params": {"color": "grey", "sele": "(protein) AND " + chaintxt, "opacity": "0.2"}})
        if len(porphtxt)>0:
            mainreps.append({"type": "licorice", "params": {"color": "green", "sele": porphtxt + ' AND ' + chaintxt, "opacity": "1.0"}})
        pdbview.set_representations(mainreps, component=0)

        if len(dipstruc.atoms)>0:
            pdbview.add_trajectory(dipstruc)
            dipreps = [{"type": "licorice", "params": {"color": "red", "sele": chaintxt, "opacity": "1", "radius": "0.35"}}]
            pdbview.set_representations(dipreps, component=1)

    # Assign tetrapyrrole types:
    # 1. Identify tetrapyrrole rings
    PigNdcs = pigment.find_pigments(tetpy.BaseNames, struc)

    # 2. Check which types are definitely excluded for each pigment
    alist = pigment.eliminate_types(PigNdcs, tetpy, TetPyList, UNK, struc)

    # 3. Now check if all xatoms of each type are present
    mlist = pigment.match_types(PigNdcs, alist, struc)
    
    # 4. Based on this data, assign pigment types
    tlist = pigment.assign_pigments(PigNdcs, mlist, alist, struc)
    
    PigList = []
    # Record pigments:
    for p in range(0, len(PigNdcs)):
        ndx = PigNdcs[p]
        res = struc.residues[ndx]
        atom_ndcs = []
        atom_names = []
        for at in res:
            atom_ndcs.append(at.idx)
            atom_names.append(at.name)
        atom_ndcs = np.array(atom_ndcs, dtype='int')
#         atcoords = struc.coordinates[atom_ndcs]

        # First dimension is frame #, second is atom number, third is x,y,z
        atcoords = struc.get_coordinates().copy()[:,atom_ndcs,:]
        PigList.append(pigment.pigment(ndx, tlist[p], alist[p], res, atom_names, atcoords))
    
    build_pigbox()
    
    chaincbs = []
    for chain in ChainList:
        chaincbs.append(widgets.Checkbox(value=True, description=chain,indent=False, layout=Layout(width='100px')))
    for cb in chaincbs:
        cb.observe(update_chains)
    chainbox.children = chaincbs
    writebt.disabled = False
    
    excgo.disabled = False
    coupselect.disabled = False
    siteselect.disabled = False
    
    if 'data/pdb/' in fname:
        prefix = fname.split('.')[-2].split('/')[-1]
    elif 'data/md/' in fname:
        prefix = fname.split('.')[-2].split('/')[-3]
    else:
        prefix = 'test'
    
    
    excfile.value = prefix
    mdfile.value = prefix
    writetxt.value = prefix+'.pdb'
    
    mdgobt.disabled = False
    waterselect.disabled = False
    ffselect.disabled = False
    

# Syncs structure display to selected chains in chainbox
def update_chains(b):
    global dipreps
    global mainreps 
    global ChainList
    
    ChainList = []
    for cb in chainbox.children:
        if cb.value==True:
            ChainList.append(cb.description)
        
    chaintxt = ''
    for chain in ChainList:
        if len(chaintxt)>0:
            chaintxt += " OR "
        chaintxt += ":" + chain
        
    # If no chain is selected, set to a nonsense chain
    # so that none will be displayed.
    if len(chaintxt)==0:
        chaintxt = ':XXXXXXXXXX'

    if NGL_DEF:
        for rep in mainreps:
            #rep['params']['sele'] = re.sub('(:[^)]+)', chaintxt, rep['params']['sele'])
            splt = rep['params']['sele'].split('(')
            newstr = ''
            for n in range(1, len(splt)-1):
                newstr += "(" + splt[n]
            newstr += '(' + chaintxt + ")"
            rep['params']['sele'] = newstr
        pdbview.set_representations(mainreps, component=0)

        #dipreps[0]['params']['sele'] = re.sub('(:[^)]+)', chaintxt, rep['params']['sele'])
        if len(dipreps)>0:
            splt = dipreps[0]['params']['sele'].split('(')
            newstr = ''
            for n in range(1, len(splt)-1):
                newstr += "(" + splt[n]
            newstr += '(' + chaintxt + ")"
            dipreps[0]['params']['sele'] = newstr
            pdbview.set_representations(dipreps, component=1)
    
pdbfetch.on_click(pdbfetch_onclick)

##################################################################
# selbox: Interface for selecting chains and residues
##################################################################

# Label for chain list
chainlbl = widgets.Label(value='Chain:')

# The children (i.e., options) of chainbox are set in pdbfetch_onclick()
# when a new structure is loaded. 
chainbox = widgets.VBox([])

selall = widgets.Button(
    description='All',
    disabled=False,
    tooltip='Click to select all chains',
    layout = widgets.Layout(width='1.5cm'),
)

selnone = widgets.Button(
    description='None',
    disabled=False,
    tooltip='Click to select all chains',
    layout = widgets.Layout(width='1.5cm'),
)

def selall_onclick(b):
    for cb in chainbox.children:
        cb.value = True
selall.on_click(selall_onclick)

def selnone_onclick(b):
    for cb in chainbox.children:
        cb.value = False
selnone.on_click(selnone_onclick)

selbox = widgets.VBox([widgets.HBox([selall, selnone]), chainlbl, chainbox])


##################################################################
# writebox: Interface for writing PDB output
##################################################################

writetxt = widgets.Text(value='test.pdb', description='File Name:', disabled=False, layout=widgets.Layout(width='5cm'))
writebt = widgets.Button(description='Write',tooltip='Click to write PDB file',layout=widgets.Layout(width='1.5cm'), disabled=True)

def pdbwrite_onclick(b):
    fname = DATADIR + "/pdb/" + writetxt.value
    if fname[-4:]!='.pdb':
        fname += '.pdb'
    
    # First identify which chains should be written
    # Loop through chain-selection check-boxes and
    # add a parmed structure object for each chain
    strucList = []
    for cb in chainbox.children:
        if cb.value==True:
            strucList.append(struc[cb.description,:,:])
    
    # Now combine all sub-structures into a single structure
    # for writing to pdb. 
    selstruc = []
    if len(strucList)>0:
        selstruc = strucList[0]
        for n in range(1, len(strucList)):
            selstruc = selstruc + strucList[n]
        selstruc.write_pdb(fname)
#         disptext = 'Structure successfully written to file <a href=\"' + fname + '\" target="_blank">'+fname+'</a>'
#         display_markdown(disptext, raw=True)
        
writebt.on_click(pdbwrite_onclick)

pdb_down = widgets.HTML(HTMLButtonPrompt.format(link=DATADIR + '/pdb/', text='Download'))

writebox = widgets.Box([widgets.HBox([writetxt, writebt]), pdb_down], 
                      layout=Layout(flex_flow='column',
                                   align_items='center'))


##################################################################
# excbox: Interface for building exciton models
##################################################################


def build_excitons(b):
    
    prefix = excfile.value
    for chars in forbidden_strings:
        if len(prefix)==0 or prefix.find(chars)!=-1:
            excoutlbl.value = 'Please enter a valid file prefix for output. Avoid spaces and the special characters "~", "..", "/", and "\\".'
            return
        
    if os.path.isdir(DATADIR + "/exc/"+prefix+".exc")==True:
        if excovercb.value==False:
            excoutlbl.value = "Directory exists. Please check 'Overwrite Existing' or pick another directory name."
            return
        else:
            !{"rm -r " + DATADIR + "/exc/"+prefix+".exc"}
            
    !{"mkdir " + DATADIR + "/exc/"+prefix+".exc"}
        
    if os.path.isdir(DATADIR + "/exc/"+prefix+".exc")==False:
        excoutlbl.value = 'Error creating directory. Please choose a different export name.'
        return
    
    if coupselect.value=='TrESP':
        CoupTraj, DipTraj, RotTraj = tresp.calculate_coupling(PigList, ChainList)
        Coups = CoupTraj[0]/len(CoupTraj)
        Dips = DipTraj[0]/len(DipTraj)
        Rots = RotTraj[0]/len(RotTraj)
        for fr in range(1, len(CoupTraj)):
            Coups += CoupTraj[fr]/len(CoupTraj)
            Dips += DipTraj[fr]/len(DipTraj)
            Rots += RotTraj[fr]/len(RotTraj)
            
        for n in range(0, np.shape(Dips)[0]):
            Dips[n,:] /= np.linalg.norm(Dips[n,:])
        
    
    
    # First set uniform frequencies (default)
    UFreqs = []
    for p in range(0, len(PigList)):
        if ChainList.count(PigList[p].residue.chain)>0:
            pname = PigList[p].species.stdname
            if pname=='CLA' or pname=='PHA':
                freq = (1.0e+7)/670.0
            elif pname=='CLB' or pname=='PHB':
                freq = (1.0e+7)/655.0
            elif pname[0]=='B':
                freq = (1.0e+7)/800.0
            else:
                freq = (1.0e+7)/670.0
            UFreqs.append(freq)
            
    # Then add shifts using TrESP method, if requested
    if siteselect.value=='TrESP':
        ShiftTraj, err = tresp.calculate_shift(PigList, ChainList, struc)
        
        # Convert to numpy array. Each row is a frame; each column is a site. 
        FTraj = np.vstack(ShiftTraj)
        
        if err:
            print('Please choose another site-energy-prediction method.')
            if opts.count('TrESP'):
                opts.remove('TrESP')
            siteselect.options = opts
            return
        else:
            # Add in the vacuum transition frequency for each pigment
            for n in range(0, len(UFreqs)):
                FTraj[:,n] += UFreqs[n]
            
        # And now average over all frames
        Freqs = np.mean(FTraj, 0)
        
        
    elif siteselect.value=='NSD':
        
        FTrajList, err = calculate_nsd(PigList, ChainList, struc)
        
        # Convert to numpy array. Each row is a frame; each column is a site. 
        FTraj = np.vstack(FTrajList)
        
        if err:
            print('Please choose another site-energy-prediction method.')
            if opts.count('NSD'):
                opts.remove('NSD')
            siteselect.options = opts
            return
            
        # And now average over all frames
        Freqs = np.mean(FTraj, 0)
        
    # If not, just use default values
    else:
        Freqs = UFreqs
        FTraj = []
                
    # If we successfully calculated coupling constants
    if len(Dips)>0:
        comment_string = 'Prepared from file: ' + struc_fname + '\n'
        comment_string += 'File included ' + str(np.shape(FTraj)[0]) + ' frames.\n'
        for p in range (0, len(PigList)):
            pig = PigList[p]
            if ChainList.count(pig.residue.chain)>0:
                comment_string += pig.residue.name + ":" + pig.residue.chain + str(pig.residue.number) + ' '
        
        np.savetxt(DATADIR + "/exc/" + prefix + '.exc/coups.txt', Coups, header = comment_string, fmt='%10.3f')
        np.savetxt(DATADIR + "/exc/" + prefix + '.exc/freqs.txt', Freqs, header = comment_string, fmt='%10.3f')
        np.savetxt(DATADIR + "/exc/" + prefix + '.exc/dips.txt', Dips, header = comment_string, fmt='%10.3f')
        np.savetxt(DATADIR + "/exc/" + prefix + '.exc/rots.txt', Rots, header = comment_string, fmt='%10.3f')
        
        # We only write a frequency trajectory if one was actually calculated. 
        if len(FTraj)>0:
            np.savetxt(DATADIR + '/exc/' + prefix + '.exc/freq_traj.txt', FTraj, header = comment_string, fmt='%10.3f')
            
        with open(DATADIR + "/exc/" + prefix + '.exc/names.txt', "w") as file:
            file.write(comment_string)
        excoutlbl.value = "Exciton model stored in directory "+ DATADIR + "/exc/"+prefix+".exc/"
        
        
couplbl = widgets.Label(value='Coupling Model')
coupselect = widgets.RadioButtons(
    options=['TrESP'],
    value='TrESP',
    disabled=True,
    layout=Layout(width='2cm')
)

sitelbl = widgets.Label(value='Site Energy Model')
siteselect = widgets.RadioButtons(
    options=['Uniform', 'NSD'],
    value='Uniform',
    disabled=True,
    layout=Layout(width='2cm')
)

excfilelbl = widgets.Label(value='File prefix: ')

excfile = widgets.Text(
    value='test',
    placeholder='file prefix',
    layout = widgets.Layout(width='2.5cm'),
    disabled=False
)

excovercb = widgets.Checkbox(value=False, description='Overwrite Existing', layout=Layout(width='4cm'))
excovercb.style.description_width='0cm'

excgo = widgets.Button(
    description='Go!',
    disabled=True,
    tooltip='Click to build an exciton model',
    layout=Layout(width='150px')
)
#excgo.style.button_color='#CCCCCC'

excgo.on_click(build_excitons)

exc_down = widgets.HTML(HTMLButtonPrompt.format(link=DATADIR + "/exc/", text='Download', layout=Layout(width='0.5cm')))

spec_button = widgets.HTML(HTMLButtonPrompt.format(link='Spectrum.ipynb', text='Simulate'))

excoutlbl = widgets.HTML(value='')

excbox = widgets.VBox(
    [
        widgets.HBox([
            widgets.VBox([couplbl, coupselect], layout=Layout(width='3cm', margin='5pt')),
            widgets.VBox([sitelbl, siteselect], layout=Layout(width='3cm', margin='5pt')),
        ], layout=Layout(width='7cm', margin='5pt'), ),
        widgets.HBox([excfilelbl, excfile]),
        excovercb,
        excgo,
        widgets.Label(layout=Layout(height='20px')),
        widgets.HBox([exc_down,spec_button]), 
        excoutlbl,
    ], layout=Layout(align_items='center', width='7.5cm'))


##################################################################
# mdbox: Interface for preparing MD models
##################################################################

fflbl = widgets.Label(value='Force Field')
ffselect = widgets.RadioButtons(
    options=['oplsaa'],
    value='oplsaa',
    disabled=True,
    layout=Layout(width='2.5cm')
)

waterlbl = widgets.Label(value='Water Model')
waterselect = widgets.RadioButtons(
    options=['spce'],
    value='spce',
    disabled=True,
    layout=Layout(width='2.5cm')
)

mdgobt = widgets.Button(
    description='Go!',
    disabled=True,
    tooltip='Click to build an MD model',
    layout=Layout(width='150px')
)

mdfilelbl = widgets.Label(value='File prefix: ')

mdfile = widgets.Text(
    value='test',
    placeholder='file prefix',
    layout = widgets.Layout(width='2.5cm'),
    disabled=False
)

mdovercb = widgets.Checkbox(value=False, description='Overwrite Existing', layout=Layout(width='4cm'))
mdovercb.style.description_width='0cm'

md_down = widgets.HTML(HTMLButtonPrompt.format(link=DATADIR + '/md/', text='Download', layout=Layout(width='0.5cm')))

mdrun_button = widgets.HTML(HTMLButtonPrompt.format(link='mdlaunch.ipynb', text='Simulate', layout=Layout(width='0.5cm')))

#mdrun_button = widgets.HTML(HTMLDeadPrompt.format(text='Simulate'))

mdoutlbl = widgets.HTML(value='')

# Get rid of all but one altloc record
def strip_altlocs(struc):
    delMask = []
    for res in struc.residues:
        al = ''
        for atom in res:
            del_flag = False

            # If entry has an altloc, let's decide what to do with it
            if len(atom.altloc.strip())>0:

                # If altloc for this residue hasn't been set, then set it
                if len(al)==0:
                    al = atom.altloc.strip()

                # Otherwise, if it doesn't match, get rid of it
                elif al!=atom.altloc.strip():
                    del_flag = True
            delMask.append(del_flag)
    struc.strip(delMask)
    return struc
    
def mdgo_onclick(b):
    protfname = "protein.pdb"
    pigfname = "pigments.pdb"
    sysfname = "system.pdb"
    
    prefix = mdfile.value
    for chars in forbidden_strings:
        if len(prefix)==0 or prefix.find(chars)!=-1:
            mdoutlbl.value = 'Please enter a valid file prefix for output. Avoid spaces and the special characters "~", "..", "/", and "\\".'
            return
        
    if os.path.isdir(DATADIR + '/md/'+prefix)==True:
        if mdovercb.value==False:
            mdoutlbl.value = "Directory exists. Please check 'Overwrite Existing' or pick another directory name."
            return
        else:
            !{"rm -r " + DATADIR + "/md/"+prefix}
            
    #mdrun_button.value = HTMLDeadPrompt.format(text='Simulate')
    !{"mkdir " + DATADIR + "/md/"+prefix}
    !{"mkdir " + DATADIR + "/md/"+prefix+"/pdb2gmx/"}
    !{"mkdir " + DATADIR + "/md/"+prefix+"/em/"}
    !{"mkdir " + DATADIR + "/md/"+prefix+"/nvt/"}
    !{"mkdir " + DATADIR + "/md/"+prefix+"/anneal/"}
#     !{"mkdir " + DATADIR + "/md/"+prefix+"/npt/"}
    !{"cp misc/pymol/mutate.py " + DATADIR + "/md/"+prefix+"/pdb2gmx/"}
    !{"cp misc/gmx/ipynb/pdb2gmx.ipynb " + DATADIR + "/md/"+prefix+"/pdb2gmx/"}
    !{"cp misc/gmx/ipynb/add_hydrogens.ipynb " + DATADIR + "/md/"+prefix+"/pdb2gmx/"}
    !{"cp " + struc_fname + " " + DATADIR + "/md/"+prefix+"/pdb2gmx/reference.pdb"}
    !{"cp misc/gmx/ipynb/parse_pqr.sh " + DATADIR + "/md/"+prefix+"/pdb2gmx/"}
    !{"cp misc/gmx/ipynb/minimize_complex.ipynb " + DATADIR + "/md/"+prefix+"/em/"}
    !{"cp misc/gmx/ipynb/md_organizer.ipynb " + DATADIR + "/md/"+prefix+"/"}
    !{"cp misc/gmx/ipynb/parse_pqr.sh " + DATADIR + "/md/"+prefix+"/em/"}
    !{"cp misc/gmx/mdp/em.mdp " + DATADIR + "/md/"+prefix+"/em/"}
    !{"cp misc/gmx/mdp/hmin.mdp " + DATADIR + "/md/"+prefix+"/pdb2gmx/"}
    !{"cp misc/gmx/mdp/nvt.mdp " + DATADIR + "/md/"+prefix+"/nvt/"}
    !{"cp misc/gmx/ipynb/run_nvt.ipynb " + DATADIR + "/md/"+prefix+"/nvt/"}
    !{"cp misc/gmx/ipynb/parse_pqr.sh " + DATADIR + "/md/"+prefix+"/nvt/"}
    !{"cp misc/gmx/mdp/anneal.mdp " + DATADIR + "/md/"+prefix+"/anneal/"}
    !{"cp misc/gmx/ipynb/run_anneal.ipynb " + DATADIR + "/md/"+prefix+"/anneal/"}
    !{"cp misc/gmx/ipynb/parse_pqr.sh " + DATADIR + "/md/"+prefix+"/anneal/"}
#     !{"cp misc/gmx/mdp/npt.mdp " + DATADIR + "/md/"+prefix+"/npt/"}
#     !{"cp misc/gmx/ipynb/npt.ipynb " + DATADIR + "/md/"+prefix+"/npt/"}
        
    if os.path.isdir(DATADIR + '/md/'+prefix)==False:
        mdoutlbl.value = 'Error creating directory. Please choose a different export name.'
        return
    
    # First identify which chains should be written
    # Loop through chain-selection check-boxes and
    # add a parmed structure object for each chain
    strucList = []
    for cb in chainbox.children:
        if cb.value==True:
            strucList.append(struc[cb.description,:,:])
    
    # Now combine all sub-structures into a single structure
    # and write to pdb. 
    selstruc = []
    if len(strucList)>0:
        selstruc = strucList[0]
        for n in range(1, len(strucList)):
            selstruc = selstruc + strucList[n]
            
        # First write total structure to system file. 
        # This file will contain all coordinates and atoms (for the selected chains)
        # *exactly* as originaly present in the PDB, including altlocs.
        selstruc.write_pdb(DATADIR + "/md/"+prefix+"/pdb2gmx/"+sysfname)
        
        # Now re-load and strip altlocs (keep only one).
        struc0 = pmd.load_file(DATADIR + "/md/"+prefix+"/pdb2gmx/"+sysfname)
        newstruc = strip_altlocs(struc0)
        
        # Write the new structure (all content) to protein file. 
        # We'll overwrite this later, after removing pigments. 
        newstruc.write_pdb(DATADIR + "/md/"+prefix+"/pdb2gmx/"+protfname)
        
        # Make a list of all pigment residue names
        ResNames = []
        StdNames = []
        for pig in PigList:
            if ResNames.count(pig.residue.name)==0:
                ResNames.append(pig.residue.name)
                StdNames.append(pig.species.stdname)
        
        # Now reload and write separate files for recognized pigments
        # and for everything else
        if len(ResNames)>0:
            struc0 = pmd.load_file(DATADIR + "/md/"+prefix+"/pdb2gmx/"+protfname)
            strucList = []
            
            # First write PDB file for pigments
            for n in range(0, len(ResNames)):
                
                name = ResNames[n]
                stdname = StdNames[n]
                
                # Create a new structure of only this pigment type
                newstruc = struc0[':'+name]
                
                # Standardize pigment names
                for res in newstruc.residues:
                    res.name = stdname
                    
                strucList.append(newstruc)
                
            # Build a combined structure including all pigments
            if len(strucList)>0:
                pigstruc = strucList[0]
                for n in range(1, len(strucList)):
                    pigstruc = pigstruc + strucList[n]
                    
                # Write this to the pigment file
                pigstruc.write_pdb(DATADIR + "/md/"+prefix+"/pdb2gmx/"+pigfname)
                
                # Check whether topologies are available
                for res in pigstruc.residues:
                    # If we don't have a topology for this pigmen on record, throw an error. 
                    if os.path.isfile('misc/gmx/itp/'+res.name+".itp")==False:
                        mdoutlbl.value = 'Could not locate topology reference file for pigment ' + res.name + ". Aborting."
                        return
                    else:
                        # If we haven't already added the top file for this pigment, add it
                        if os.path.isfile(DATADIR + "/md/"+prefix+"/pdb2gmx/"+res.name+".itp")==False:
                            out = !{"cp misc/gmx/itp/"+res.name+".itp "+ DATADIR + "/md/"+prefix+"/pdb2gmx/"}
                
            # Now rewrite the main file without pigments
            for n in range(0, len(ResNames)):
                name = ResNames[n]
                struc0.strip(':'+name)
                
            # This file will contain everything *not* recognized as a pigment. 
            struc0.write_pdb(DATADIR + "/md/"+prefix+"/pdb2gmx/"+protfname)
    
    with open(DATADIR + "/md/"+prefix+"/pdb2gmx/ffparams.txt", 'w') as fd:
        fd.write('FF: ' + ffselect.value + '\n')
        fd.write('WATER: ' + waterselect.value + '\n')
    mdoutlbl.value='MD inputs writen to directory ' + prefix + '.'
    #mdrun_button.value = HTMLButtonPrompt.format(link=DATADIR + "/md/"+prefix+'/md_organizer.ipynb', text='Simulate')
    
    # Write description to log file 
    with open(DATADIR + "/md/"+prefix+"/pdb2gmx/logfile.txt", 'w') as fd:
        fd.write('Structures in this folder were prepared from input file ' + struc_fname + '.\n')
        fd.write('This original structure has been copied to reference.pdb.\n')
        fd.write('Selected chains (all content, including altlocs) have been written to system.pdb.\n')
        fd.write('Pigments (with altlocs stripped) have been written to pigments.pdb.\n')
        fd.write('Non-Pigments (with altlocs stripped) have been written to protein.pdb.\n')
    

mdgobt.on_click(mdgo_onclick)

mdbox = widgets.Box([widgets.HBox([
            widgets.VBox([fflbl, ffselect], layout=Layout(width='3cm', margin='5pt')),
            widgets.VBox([waterlbl, waterselect], layout=Layout(width='3cm', margin='5pt')),
        ], layout=Layout(width='6cm', margin='5pt'), ),
        widgets.HBox([mdfilelbl, mdfile]),
        mdovercb,
        mdgobt, 
        widgets.HBox([md_down, mdrun_button]),
        mdoutlbl], 
    layout=Layout(flex_flow='column',align_items='center'))



#######################################################
# Main Box
#######################################################


def update_pigtypes(change):
    global PigList
    
    for pig in PigList:
        if pig.widget.value!=pig.species.name:
            for typ in pig.alist:
                if typ.name==pig.widget.value:
                    pig.species = typ
                    break

def build_pigbox():
    global pigbox
    global PigList
    
    pigbox.layout = Layout(display='flex',
                    flex_flow='row wrap',
                    align_items='stretch',
                    width='100%')

    hlist = []
    for chain in ChainList:
        chwidglist = []
        for p in range(0, len(PigList)):
            pig = PigList[p]
            res = struc.residues[pig.idx]
            if res.chain==chain:
                dropbox = widgets.Dropdown(
                            options=[typ.name for typ in pig.alist],
                            value=pig.species.name,
                            description=res.name + ' ' + chain + str(res.number),
                            disabled=False,
                            layout=Layout(width='4.5cm')
                        )
                dropbox.observe(update_pigtypes, 'value')
                pig.widget = dropbox
                chwidglist.append(dropbox)
        if len(chwidglist)>0:
            lbl = widgets.Label(value='Chain ' + chain + ":")
            hlist.append(widgets.VBox([lbl] + chwidglist))
    pigbox.children = hlist

mainacc = widgets.Accordion(children=[strucbox, selbox, writebox, excbox, mdbox], layout=Layout(width='8cm'))
mainacc.set_title(0, 'Load')
mainacc.set_title(1, 'Select')
mainacc.set_title(2, 'Write PDB')
mainacc.set_title(3, 'Build Exciton Model')
mainacc.set_title(4, 'Build MD Model')

# def on_accordion_change(change):
#     if change["new"]!=4:
#         time.sleep(0.1)
#         #mdrun_button.value = HTMLDeadPrompt.format(text='Simulate')
    
#mainacc.observe(on_accordion_change, names="selected_index")

pigbox = widgets.HBox()
pigacc = widgets.Accordion(children=[pigbox])
pigacc.set_title(0, 'Pigment List')

mainbox = widgets.VBox([widgets.HBox([pdbview, mainacc]), pigacc])
display(mainbox)

excfile.value = '2DRE'
writetxt.value = '2DRE.pdb'
mdfile.value = '2DRE'



In [6]:
def nsd_transformation(refxyz, coordinates):
        
    # non weighted center of mass
    rcm = np.sum(coordinates,0)/np.shape(refxyz)[0]
    
    # shift
    xyz = coordinates - rcm
    
    # Each elements of U should contain a sum over atom indices. 
    # a and b iterate over coordinates (x,y,z).
    # The sum over atoms is accomplished using np.dot(). 
    U = np.zeros((3,3))
    for a in range(0,3):
        for b in range(0,3):
            U[a,b] = np.dot(xyz[:,a], xyz[:,b])
    
    # Passing two output arguments means that the first will be eigenvalues and the second eigenvectors
    evals, evecs = np.linalg.eig(U)
    
    # This orders all eigenvalues from smallest to largest. 
    # This isn't really needed here, but it's often useful. 
    # Note that the nth *column* (not row) of evecs is the 
    # eigenvector corresponding to the nth eigenvalue. 
    idx = evals.argsort()
    sevals = evals[idx]
    sevecs = evecs[:,idx]
    m = sevecs[:,0]
    if np.dot(np.cross(xyz[1,:], xyz[0,:]), m)<0:
        m = -m

        
    ##############
    # Rotation 1 #
    ##############
    
    # Rotation axis unit vector
    u = np.cross(m, np.array([0,0,1]))/np.linalg.norm(np.cross(m, np.array([0,0,1])))
    
    # angle
    b = np.arccos(m[2])
    
    # Transformation object
    R = spatial.transform.Rotation.from_rotvec(u*(b))
    newxyz = R.apply(xyz)
    
    
    ##############
    # Rotation 2 #
    ##############
    
    # Calculate theta
    top = (np.dot(refxyz[:,0], newxyz[:,1])-np.dot(refxyz[:,1], newxyz[:,0]))
    bot = (np.dot(refxyz[:,0], newxyz[:,0])+np.dot(refxyz[:,1], newxyz[:,1]))
    theta0 = np.arctan(top/bot)
    
    # Keep between 0 and 2*pi
    if theta0 < 0:
        theta1 = theta0 + math.pi
        theta2 = theta0 + 2.0*math.pi
    else:
        theta1 = theta0
        theta2 = theta0 + math.pi
    
    # Now distinguish maximum vs. minimum. (Shifted by pi from each other.)
    # Note typo in paper!
    discrim1 = (np.dot(refxyz[:,0], newxyz[:,1])-np.dot(refxyz[:,1], newxyz[:,0]))*np.sin(theta1) + (np.dot(refxyz[:,0], newxyz[:,0])+np.dot(refxyz[:,1], newxyz[:,1]))*np.cos(theta1)
    discrim2 = (np.dot(refxyz[:,0], newxyz[:,1])-np.dot(refxyz[:,1], newxyz[:,0]))*np.sin(theta2) + (np.dot(refxyz[:,0], newxyz[:,0])+np.dot(refxyz[:,1], newxyz[:,1]))*np.cos(theta2)
    
    # b1 is the rotation angle
    if discrim1>0:
        b1 = theta1
    elif discrim2>0:
        b1 = theta2
    
    # check if values make sense
    if (discrim1>0) and (discrim2)>0:
        print('Error! Both rotation angles were positive!')
    if (discrim1<0) and (discrim2)<0:
        print('Error! Neither rotation angle was positive!')
        
    # Rotation axis unit vector
    u1 = np.array([0,0,1])
    
    # Transformation object
    R1 = spatial.transform.Rotation.from_rotvec(u1*(-b1))
    newnewxyz = R1.apply(newxyz)
    
    return newnewxyz-refxyz




# Dobs is the difference between refxyz coordinates and pigment coordinates
# after rotation into the NSD reference frame. 
def nsd_calc(Dobs, Dgamma):
    
    Nrows = np.shape(Dgamma)[0]
    Ncols = np.shape(Dgamma)[1]
    
    # Deformation 
    D0 = []
    OOP = []
    
    # G loops through symmetry groups. 
    # The first few symmetry groups have three independent vectors
    for G in range(0, Ncols-2, 3):
        Dvecs = Dgamma[:,G:G+3]
        
        # Minimal basis calculation
        P = np.zeros((3,))
        for j in range(0,3):
            P[j] = np.dot(Dvecs[:,j], Dobs[:,2])
        D0.append(P[0])
        
        # Full basis calculation
        B = np.zeros((3,3))
        for j in range(0,3):
            for m in range(0,3):
                B[j,m] = np.dot(Dvecs[:,j], Dvecs[:,m])
        
        Binv = np.linalg.inv(B)
        dvec = Binv@P
        dtot = np.sqrt(dvec.transpose()@B@dvec)
        OOP.append(dtot)
        
    # The last symmetry group has only two vectors
    # NB: Note hard-coded column numbers. :(
    Dvecs = Dgamma[:,15:17]
    
    # Mimimal basis
    P = np.zeros((2,))
    for j in range(0,2):
        P[j] = np.dot(Dvecs[:,j], Dobs[:,2])
    D0.append(P[0])
    
    # Full basis
    B = np.zeros((2,2))
    for j in range(0,2):
        for m in range(0,2):
            B[j,m] = np.dot(Dvecs[:,j], Dvecs[:,m])
            
    Binv = np.linalg.inv(B)
    dvec = Binv@P
    dtot = np.sqrt(dvec.transpose()@B@dvec)
    OOP.append(dtot)
    
    return np.array(D0)


def siteenergy(d0, pars):
    
    # other constants
    h = 6.626e-37 #kJ*s
    c = 2.998e10 #cm/sec

    #out of plane displacements from table
    ED = 0

    #force constant (1/((cm*A^2))
    k = pars[0:6]
    
    # H --> L gap
    a = pars[6]
    
    # H-1 --> L+1 gap
    b = pars[7]
    
    # Coupling
    C = pars[8]
    
    for n in range(0,6):
        ED += k[n] * d0[n]**2
        
    X1 = a + k[3]*(d0[3]**2) - k[5]*(d0[5]**2) # frequency of H to L transition (cm-1)
    X2 = b + k[4]*(d0[4]**2) - k[2]*(d0[2]**2) # frequency of H-1 to L+1 transition (cm-1)
    E = 0.5*( (X1+X2) - math.sqrt( (X2-X1)**2 + (4*(C**2)) )) # site energy of deformation energy in kJ/mol 
    
    return E
        
def calculate_nsd(PigList, ChainList, instruc):
    
    # First create list of chain-selected pigments
    SelPigs = []
    for pig in PigList:
        if ChainList.count(pig.residue.chain)>0:
            SelPigs.append(pig)
    
    Npigs = len(SelPigs)
    FreqTraj = []
        
    h = 6.62607015e-34 #J*s
    c = 2.998e10 #cm/s
    eo = 4.80320451e-10 # esu
    eps_eff = 2.5
    
    Erg2J = 1.0e-7
    
    # Now check whether NSD parameters are available for *all*
    # selected pigments and (if so) store them for reference. 
    
    # Loop over pigments
    error = False
    ListNames = []
    ListPar = []
    ListRefXYZ = []
    ListDGamma = []
    for p in range(0, Npigs):
        pig = SelPigs[p]
        
        # NSD parameters should be in two separate files:
        #    'misc/NSD/' + pig.species.stdname + '/names.txt' stores atom names
        #         -- the first column is the PDB standard, second is NSD convention
        #    'misc/NSD/' + pig.species.stdname + '/dgamma.txt' stores the DGamma matrix
        #    'misc/NSD/' + pig.species.stdname + '/xyz.txt' stores xyz reference structure
        #    'misc/NSD/' + pig.species.stdname + '/par.txt' stores mixing parameters
        #         -- Columns 0 - 5 are force constants. Columns 6 - 8 are H --> L gap, H-1 --> L+1 gap, and coupling term. 
        fpar = 'misc/NSD/' + pig.species.stdname + '/par.txt'
        fxyz = 'misc/NSD/' + pig.species.stdname + '/xyz.txt'
        fdgamma = 'misc/NSD/' + pig.species.stdname + '/dgamma.txt'
        fnsdnms = 'misc/NSD/' + pig.species.stdname + '/names.txt'
        
        flist = [fnsdnms, fdgamma, fxyz, fpar]
        for fnm in flist:
            if os.path.isfile(fnm)==False:
                print('Error: Could not locate NSD file ' + fnm)
                error = True
                break
                
        # If we found all the files, load the parameters and store in the appropriate List
        if not error:
            
            ListPar.append(np.loadtxt(fpar))
            ListRefXYZ.append(np.loadtxt(fxyz))
            ListDGamma.append(np.loadtxt(fdgamma)*(1.0e-4))
            
            atnms = []
            with open(fnsdnms) as fd:
                for line in fd:
                    if len(line)>0 and line[0]!="#":
                        lst = line.split()
                        if len(lst)<2:
                            print('Error reading NSD input file ' + fnsdnms + '.')
                            error = True
                            break
                        else:
                            atnms.append([lst[0], lst[1]])
                            
            # Check that new XYZ and DGamma additions are of the same length as new name list.
            # Number of rows in DGamma and RefXYZ should be the same as len(atnms). 
            natoms = len(atnms)
            if natoms==np.shape(ListRefXYZ[p])[0] and natoms==np.shape(ListDGamma[p])[0]:
                
                # Note that this is a list of tuple-lists, not a NumPy array
                ListNames.append(atnms)
            else:
                error = True
                print('Error: NSD name file ' + fnsdnms + ' appears to reference a different number (' 
                      + str(natoms) + ') than the corresponding DGamma (' + str(np.shape(ListDGamma[p])[0]) + ') and/or refxyz file (' + str(np.shape(ListRefXYZ[p])[0]) + ')')
                
        # If there's an error, we break out of the loop over pigments. 
        # If not, proceed to the next pigment
        if(error):
            break
                            
    # If no errors, we've located all relevant parameter files and imported the data. 
    # Now check whether all necessary atoms are available in the structure.
    if error==False:
        for p in range(0, Npigs):
            
            pig = SelPigs[p]
            
            # Loop over atom names. PDB std names are in first column, NSD names in second. 
            for n in range(0, len(ListNames[p])):

                # PDB name for the nth NSD atom in pth pigment
                name = ListNames[p][n][0]

                # If we can't find it in the structure, throw an error. 
                # pig.atnames stores the names of all atoms associated with pigment pig
                if pig.atnames.count(name)==0:
                    print('Error: Could not locate NSD atom ' + name + " in pigment " + pig.residue.name + " " + pig.residue.chain + " " + str(pig.residue.number))
                    print('Aborting NSD calculation.')
                    error = True
                    break

            # If there's already been an error, don't continue. 
            if error:
                break
                
    # If no errors, all pigments have NSD parameters and necessary atoms. 
    if error==False:

        # Number of frames in trajectory
        Nframes = np.shape(SelPigs[0].atcoords)[0]

        # Build list of NSD coordinates for each pigment
        # NB: Units are converted to cm! (cgs units)
        CoordList = []
        for p in range(0, Npigs):
            pig = SelPigs[p]
            coords = np.zeros((Nframes, len(ListNames[p]), 3))
            
            # Find index associated with each NSD atom in the structure.
            # Coordinate list will be sorted in atom order specified in NSD
            # name file *not* order of crystal structure. 
            for n in range(0, len(ListNames[p])):
                name = ListNames[p][n][0]
                ndx = pig.atnames.index(name)
                coords[:,n,:] = pig.atcoords[:,ndx,:].copy()
            CoordList.append(coords)
            
        # Coordinates for the nth NSD atom in fth frame are in CoordList[f,n,:]
        for fr in range(0, Nframes):

            tFreqs = np.zeros((Npigs,))

            for p in range(0, Npigs):
                
                pig = SelPigs[p]
                shift = 0.0
                
                # These are coordinates for the NSD atoms in the current frame
                XYZ = CoordList[p][fr,:,:]
                
                # Transform to reference basis
                Dobs = nsd_transformation(ListRefXYZ[p], XYZ)
                
                # Run NSD calculation
                d0 = nsd_calc(Dobs, ListDGamma[p])
                
                
                # Calculate site energy 
                tFreqs[p] = siteenergy(d0, ListPar[p])
                
            FreqTraj.append(tFreqs)
        
    return FreqTraj, error
